# Normalize the data

- dates
- split products
- split ammounts
- split prices
- prices with coma

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("../data/inital_dataset.csv")

In [2]:
# Dates
df['Fecha'] = pd.to_datetime(df['Fecha'], format='mixed', dayfirst=True)
df.to_csv('../data/data_norm.csv', index=False)

# Product table

In [3]:
# Product table
productList = pd.DataFrame(df['Productos'].str.split(';').explode().str.strip().unique())
productList = productList.reset_index(drop=False)
productList.columns = ['id', 'Name']
productList['id'] += 1
productList.head(8)

,id,Name
0,1,Batería 12V
1,2,Cable 2m
2,3,Aceite lubricante
3,4,Filtro aire
4,5,Compresor portátil
5,6,Taladro 500W
6,7,Llave inglesa
7,8,Kit reparación


In [4]:
productList.to_csv('../exports/product.csv', index=False)

# ProductTransaction Table

In [5]:
df_temp = df.copy()
df_temp['TransactionID'] = df_temp.index

productos = pd.DataFrame({
    'TransactionID': df_temp['TransactionID'].repeat(df_temp['Productos'].str.split(';').str.len()),
    'ProductName': df['Productos'].str.split(';').explode().str.strip(),
    'Quantity': df['Cantidades'].str.split(';').explode().str.strip(),
    'Price': df['Precios'].str.split(';').explode().str.strip()
})
productos['Quantity'] = pd.to_numeric(productos['Quantity'])
productos['Price'] = pd.to_numeric(productos['Price'].str.replace(',', '.'))
productos.head(8)

,TransactionID,ProductName,Quantity,Price
0,0,Batería 12V,8,61.21
1,1,Cable 2m,4,32.22
1,1,Aceite lubricante,8,4.21
1,1,Filtro aire,8,62.28
2,2,Aceite lubricante,4,36.32
2,2,Cable 2m,5,3.98
3,3,Compresor portátil,9,3.37
3,3,Aceite lubricante,8,54.08


In [6]:
# map products
products_id_map = dict(zip(productList['Name'], productList['id']))

productos['ProductID'] = productos['ProductName'].map(products_id_map)
productos = productos.drop('ProductName',axis=1)
productos.head(8)

,TransactionID,Quantity,Price,ProductID
0,0,8,61.21,1
1,1,4,32.22,2
1,1,8,4.21,3
1,1,8,62.28,4
2,2,4,36.32,3
2,2,5,3.98,2
3,3,9,3.37,5
3,3,8,54.08,3


In [7]:
productos.reset_index()
productos['id'] = range(0, len(productos))
productos['id'] += 1
productos['TransactionID'] += 1
productos

,TransactionID,Quantity,Price,ProductID,id
0,1,8,61.21,1,1
1,2,4,32.22,2,2
1,2,8,4.21,3,3
1,2,8,62.28,4,4
2,3,4,36.32,3,5
...,...,...,...,...,...
197,198,9,50.58,8,510
198,199,9,59.07,5,511
198,199,1,67.56,2,512
199,200,8,61.79,5,513


In [8]:
productos.to_csv('../exports/ProductTransaction.csv', index=False)

## Update the TotalPrice value for that Transaction

In [9]:
transactions = pd.read_csv("../exports/transaction.csv")
productos['Quantity'] = pd.to_numeric(productos['Quantity'], errors='coerce')
productos['Price'] = pd.to_numeric(productos['Price'], errors='coerce')

# Calculate line totals
productos['LineTotal'] = productos['Quantity'] * productos['Price']
transaction_totals = productos.groupby('TransactionID')['LineTotal'].sum()

for idx in transactions.index:
    current_total = transactions.at[idx, 'TotalPrice']
    
    # If TotalPrice is empty, null, or 0, use calculated value
    if pd.isna(current_total) or current_total == '' or current_total == 0:
        if idx in transaction_totals.index:
            transactions.at[idx, 'TotalPrice'] = transaction_totals[idx]
transactions.to_csv('../exports/transaction.csv', index=False)